---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [9]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

def get_City_Zhvi_AllHomes():
    
    states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}
    file = 'City_Zhvi_AllHomes.csv'
    housing = pd.read_csv(file)
    housing.replace({'State':states}, inplace = True)
    housing = housing.sort_values(by = ['State','RegionName'], ascending=[True,True])
    return housing

get_City_Zhvi_AllHomes()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
5344,10040,Adamsville,Alabama,Birmingham,Jefferson,5345,60200.0,60800.0,61000.0,60500.0,...,73900,73500,73600,74100,74800,75200,74900,74700,74700,74700
1680,43839,Alabaster,Alabama,Birmingham,Shelby,1681,108400.0,108300.0,107800.0,107500.0,...,151800,152400,152800,153400,154200,154900,155200,155200,155600,156100
1561,8332,Albertville,Alabama,Albertville,Marshall,1562,67500.0,67200.0,66900.0,66200.0,...,91600,91500,91400,92000,92600,92600,92400,92400,92300,92100
3087,35038,Arab,Alabama,Albertville,Marshall,3088,75000.0,74600.0,74400.0,73900.0,...,112500,112100,111000,109800,109300,109500,110100,110700,111600,112400
5433,57107,Ardmore,Alabama,Huntsville,Madison,5434,NaN,NaN,NaN,NaN,...,144200,143100,142200,142500,143000,143200,143400,143100,142300,141600
8799,23564,Axis,Alabama,Mobile,Mobile,8800,NaN,NaN,NaN,NaN,...,109500,109000,109600,110300,110900,111700,112300,112600,112800,112700
9914,44007,Baileyton,Alabama,Cullman,Cullman,9915,NaN,NaN,NaN,NaN,...,97200,97400,97200,96600,96500,97500,99100,100100,101300,103100
3030,3562,Bay Minette,Alabama,Daphne,Baldwin,3031,66300.0,66500.0,68100.0,69400.0,...,121700,122000,121300,120300,119200,118500,118300,118200,118300,118700
9801,10284,Bayou La Batre,Alabama,Mobile,Mobile,9802,41000.0,40300.0,40100.0,39600.0,...,48900,48600,48700,48700,48300,47800,47800,47900,47600,47200
2125,16997,Bessemer,Alabama,Birmingham,Jefferson,2126,NaN,NaN,NaN,NaN,...,59500,59600,59600,59800,59900,59700,59900,60000,59900,59700


In [10]:
def get_gdplev():
    
    file2 = 'gdplev.xls'
    xls = pd.ExcelFile(file2)
    GDP = xls.parse(0, skiprows = 220, header = None)
    GDP = GDP.drop(GDP.columns[[0,1,2,3,6,7]], axis = 1)
    GDP.rename(columns = {4:'Quarters',5:'GDP'}, inplace = True)
    return GDP

get_gdplev()

,Quarters,GDP
0,2000q1,10031.0
1,2000q2,10278.3
2,2000q3,10357.4
3,2000q4,10472.3
4,2001q1,10508.1
5,2001q2,10638.4
6,2001q3,10639.5
7,2001q4,10701.3
8,2002q1,10834.4
9,2002q2,10934.8


In [11]:
def get_list_of_university_towns():
    
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    fhand = open('university_towns.txt') 
    university = pd.read_table('university_towns.txt', sep="\n",names=["States"])
    university['State'] = university.States.str.extract('(.*)\[edit\]', expand=False).ffill()
    university['RegionName'] = university.States.str.replace(r'\s+\(.+$', '')
    university = university[~university.States.str.contains('\[edit\]')].reset_index(drop=True).drop('States', axis=1)
    return university

get_list_of_university_towns()

,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [12]:
def join_tables():
    
        inner = pd.merge(get_list_of_university_towns(), get_City_Zhvi_AllHomes(), how = "inner", left_on = ['RegionName','State'], right_on = ['RegionName','State'])
        return inner.head()
    
join_tables()

,State,RegionName,RegionID,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,Alabama,Montevallo,19437,Birmingham,Shelby,3340,85600.0,85900.0,85500.0,84600.0,...,119700,120200,121000,121800,122300,122600,122500,122100,121600,121400
1,Alabama,Tuscaloosa,14208,Tuscaloosa,Tuscaloosa,288,95800.0,95600.0,95700.0,95500.0,...,145300,144400,142100,139900,138900,138500,137800,137800,138400,138900
2,Alaska,Fairbanks,38465,Fairbanks,Fairbanks North Star,839,NaN,NaN,NaN,NaN,...,209700,209600,208800,208700,209700,211200,213000,214600,215600,216100
3,Arizona,Flagstaff,31567,Flagstaff,Coconino,388,140700.0,141000.0,141500.0,142800.0,...,310100,310500,310800,311900,314100,316600,318500,320200,322000,323300
4,Arizona,Tempe,47958,Phoenix,Maricopa,153,95400.0,95900.0,96300.0,96700.0,...,237900,239700,240800,241900,243000,244000,245000,246000,246700,247200


In [13]:
def get_recession_start():

    GDP = get_gdplev()
    quarter = []
    for i in range(len(GDP) -2):
        if (GDP.iloc[i]['GDP'] > GDP.iloc[i+1]['GDP']) & (GDP.iloc[i+1]['GDP'] > GDP.iloc[i+2]['GDP']):
            quarter.append(GDP.iloc[i]['Quarters'])
    return quarter[0]
    
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''

get_recession_start()

'2008q3'

In [57]:
def get_recession_end():
    
    GDP = get_gdplev()
    quarter = []
    for i in range(len(GDP) -4):
        if (GDP.iloc[i]['GDP'] > GDP.iloc[i+1]['GDP']) & (GDP.iloc[i+1]['GDP'] > GDP.iloc[i+2]['GDP']) & (GDP.iloc[i+2]['GDP'] < GDP.iloc[i+3]['GDP']) & (GDP.iloc[i+3]['GDP'] < GDP.iloc[i+4]['GDP']):
            quarter.append(GDP.iloc[i+4]['Quarters'])
    return quarter[0]

    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
get_recession_end()

'2009q4'

In [58]:
def get_recession_bottom():
    
    GDP = get_gdplev()
    quarter = []
    for i in range(len(GDP) -3):
        if (GDP.iloc[i]['GDP'] >= GDP.iloc[i+1]['GDP']):
            quarter.append(GDP.iloc[i+1])
    return quarter[3][0]

    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
get_recession_bottom()

'2009q2'

In [59]:
#def plot_01():
    #import matplotlib as plt
    #%matplotlib inline
    
    #GDP = get_gdplev()
    #GDP.plot(x = 'Quarters', y = 'GDP', kind = "line")
    
#plot_01()

In [60]:
from datetime import datetime
from dateutil.parser import parse
import pandas as pd

def convert_housing_data_to_quarters():
    
    housing = get_City_Zhvi_AllHomes()
    GDP = get_gdplev()
    housing = housing.iloc[:, [2,1] + list(range(51, 251))]
    columns_name = []
    columns_values = []
    housing = housing.reset_index().drop('index', axis=1)
    housing = housing.set_index(['State','RegionName'])
    housing.columns = pd.to_datetime(housing.columns)
    housing = housing.resample('Q',axis=1).mean()
    housing = housing.rename(columns=lambda x: str(x.to_period('Q')).lower())
    return housing.head()
    
    
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    
convert_housing_data_to_quarters()

2000q1         2000q2         2000q3  \
State     RegionName                                                     
Alabama   Adamsville        69033.333333   69166.666667   69800.000000   
          Alabaster        122133.333333  123066.666667  123166.666667   
          Albertville       73966.666667   72600.000000   72833.333333   
          Arab              83766.666667   81566.666667   81333.333333   
          Ardmore                    NaN            NaN            NaN   
          Axis                       NaN            NaN            NaN   
          Baileyton                  NaN            NaN            NaN   
          Bay Minette       81700.000000   78533.333333   79133.333333   
          Bayou La Batre    44066.666667   44500.000000   44266.666667   
          Bessemer                   NaN            NaN            NaN   
          Birmingham        54033.333333   54400.000000   54966.666667   
          Boaz              70866.666667   70266.666667   70300.000000   
          Brent             92933.333333   94333.333333   96166.666667   
          Brighton                   NaN            NaN            NaN   
          Brookwood         92566.666667   95100.000000   98866.666667   
          Buhl              90800.000000   94600.000000   96500.000000   
          Calera           108933.333333  110366.666667  108000.000000   
          Center Point      80966.666667   81233.333333   81500.000000   
          Centreville       95300.000000   96566.666667   98000.000000   
          Chalkville        94100.000000   94433.333333   94433.333333   
          Chancellor                 NaN            NaN            NaN   
          Chelsea          162066.666667  167033.333333  166900.000000   
          Chickasaw         51200.000000   53666.666667   54933.333333   
          Chunchula         80266.666667   81766.666667   82200.000000   
          Citronelle        64833.333333   66633.333333   68066.666667   
          Clay             120900.000000  122266.666667  123966.666667   
          Coden             62600.000000   64800.000000   66866.666667   
          Coker            118100.000000  120766.666667  118166.666667   
          Concord           78600.000000   78700.000000   80133.333333   
          Cottondale       100833.333333  102633.333333  104766.666667   
...                                  ...            ...            ...   
Wisconsin Vernon           183200.000000  178200.000000  174300.000000   
          Vienna           178033.333333  181533.333333  182433.333333   
          Vinland          119800.000000  126766.666667  134933.333333   
          Wales                      NaN            NaN            NaN   
          Waterford        121200.000000  119433.333333  120200.000000   
          Waukesha         141266.666667  138433.333333  136733.333333   
          Waunakee         187400.000000  191433.333333  192666.666667   
          Waupun            84000.000000   84000.000000   84400.000000   
          Wausau            69566.666667   69466.666667   71033.333333   
          Wayne                      NaN            NaN            NaN   
          West Allis        80933.333333   81933.333333   84066.666667   
          Weston            80166.666667   82700.000000   84166.666667   
          Wheatland        129800.000000  132133.333333  134933.333333   
          Whitelaw          96033.333333  101433.333333  108033.333333   
          Williams Bay     122900.000000  115300.000000  110766.666667   
          Wilson           129033.333333  129366.666667  132433.333333   
          Wilson                     NaN            NaN            NaN   
          Wind Lake        124366.666667  123666.666667  124133.333333   
          Wind Point       149233.333333  145266.666667  140866.666667   
          Wisconsin Dells  100000.000000  103400.000000  105000.000000   
          Wolf River        95166.666667   99333.333333  103800.000000   
          Woodruff                   N

In [61]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
def run_ttest():
    new_data = convert_housing_data_to_quarters().copy()
    university = get_list_of_university_towns()
    university = university.set_index(['State','RegionName'])
    university = university.index.tolist()
    new_data = new_data.loc[:,'2008q2':'2009q2']
    new_data['ratio'] = (new_data['2008q2'] - new_data['2009q2'])/new_data['2008q2']
    group_Utown = new_data.loc[university]
    group_town = new_data.loc[-new_data.index.isin(university)]
    # possible d'ajouter .dropna(how = 'all')
    ttest = ttest_ind(group_Utown['ratio'], group_town['ratio'], nan_policy = 'omit')
        
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return (True, ttest[1], 'university town')

run_ttest()

/opt/conda/lib/python3.6/site-packages/ipykernel/__main__.py:12: DeprecationWarning: numpy boolean negative, the `-` operator, is deprecated, use the `~` operator or the logical_not function instead.


(True, 0.0033629228768433245, 'university town')